In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import networkx
import random
import math
import matplotlib.pyplot
import seaborn
import os

# cadCAD specific libraries
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [ ]:
# Make a folder in the working directory for storing visualization files
if not os.path.exists('results_encointer_simulation'):
  os.mkdir('results_encointer_simulation')

In [ ]:
# Fixing the randomness
random.seed(1)
np.random.seed(1)

### Helper Functions

In [ ]:
# gini index

def gini(graph, account_type):
    accounts = [graph.nodes[person][account_type] for person in graph.nodes]
    sorted_accounts = sorted(accounts)

    #total wealth of the community
    total_wealth = sum(sorted_accounts)

    #no division zero, perfect equality
    if total_wealth == 0:
        return 0

    #each agent represents a fraction of the total population
    agent_fraction = 1/len(sorted_accounts)

    #cumulative portion of the total wealth
    cumulative_wealth_portion = 0

    #area under the lorenz curve
    area_lorenz_curve = 0

    for person in range(len(sorted_accounts)):

        #LORENZ CURVE OF DISCRETE WEALTH DISTRIBUTION

        previous_cumulative_wealth_portion = cumulative_wealth_portion

        #normalized wealth share of person
        wealth_share_of_person = sorted_accounts[person]/total_wealth

        #cumulative portion of the total wealth covered until now
        cumulative_wealth_portion += wealth_share_of_person

        #AREA UNDER THE LORENZ CURVE

        rectangle = agent_fraction * cumulative_wealth_portion
        triangle = agent_fraction * (cumulative_wealth_portion - previous_cumulative_wealth_portion) / 2
        trapeze = rectangle - triangle

        area_lorenz_curve+= trapeze

    # gini index
    gini = (0.5 - area_lorenz_curve) / 0.5

    if gini < 0:
        gini = 0

    return gini

In [ ]:
# Demurrage

def demurrage(demurrage_half_life):

    # Demurrage rate
    demurrage_rate = np.exp(np.log(0.5)/demurrage_half_life)

    return 1-demurrage_rate

In [ ]:
# Logistic growth

def expit(x):
    y = 1 / (1 + np.exp(-x))
    return y

def logit(x):
    return np.log(x/(1-x))

def logistic_growth(N, K, delta_x = 1):
    if N == K:
      return 0
    elif N == K-1:
      return 1
    else:
      return math.floor((expit(logit(N/K) + delta_x) - N/K) * K)

In [ ]:
# Probability distribution of an agent to attend the ceremony as a function of wealth

def find_probability_ceremony_participant(agent_wealth, max_wealth, param_scale = 0.75, param_translate = 1.5):
    if max_wealth == 0:
      return 1
    half_wealth = max_wealth/2
    scale_value = 6*param_scale/half_wealth
    translate_value = half_wealth+param_translate
    return 1-expit(scale_value*(agent_wealth-translate_value))

# Plot
matplotlib.pyplot.figure('Probability Distribution of Attending Ceremony')
x = np.linspace(0, 6, 121)
y = find_probability_ceremony_participant(x, 6)
matplotlib.pyplot.plot(x, y)
matplotlib.pyplot.xlim(0, 6)
matplotlib.pyplot.ylim(0, 1)
matplotlib.pyplot.xlabel('Wealth [LEU+CHF*XR]')
matplotlib.pyplot.ylabel('Probability')
matplotlib.pyplot.title('Probability Distribution of Attending Ceremony')
matplotlib.pyplot.savefig('results_encointer_simulation/Probability Distribution of Attending Ceremony', dpi=300)

In [ ]:
# Probability distribution of an agent to be selected as buyer as a function of wealth

def find_probability_buyer(agent_wealth, max_wealth, param_scale = 1, param_translate = -1):
    if max_wealth == 0:
      return 1
    half_wealth = max_wealth/2
    scale_value = 6*param_scale/half_wealth
    translate_value = half_wealth+param_translate
    return expit(scale_value*(agent_wealth-translate_value))

# Plot
matplotlib.pyplot.figure('Probability Distribution of Purchasing Products')
x = np.linspace(0, 6, 121)
y = find_probability_buyer(x, 6)
matplotlib.pyplot.plot(x, y)
matplotlib.pyplot.xlim(0, 6)
matplotlib.pyplot.ylim(0, 1)
matplotlib.pyplot.xlabel('Wealth')
matplotlib.pyplot.ylabel('Probability')
matplotlib.pyplot.title('Probability Distribution of Purchasing Products')
matplotlib.pyplot.savefig('results_encointer_simulation/Probability Distribution of Purchasing Products', dpi=300)

In [ ]:
# Generating new agents and inserting into graph

def generate_new_agents(graph, amount_new_people, basic_income, percentage_of_classes, lower_class_chf, middle_class_chf, upper_class_chf, exchange_rate):
    last_person_index = len(list(graph.nodes))
    amount_new_people = int(str(amount_new_people))

    for person in range(last_person_index, last_person_index+amount_new_people):

        # Give national currency according to social class
        money_distribution_in_population = [lower_class_chf, middle_class_chf, upper_class_chf]
        index = list(range(len(money_distribution_in_population)))
        probabilities_money_distribution_in_population = percentage_of_classes #must sum to 1
        index_chosen = np.random.choice(index , p=probabilities_money_distribution_in_population)
        person_chf = random.randint(*money_distribution_in_population[index_chosen])

        # Set initial status of agent according to national currency account; lower: 0, middle: 1, upper: 2
        person_status_initial = index_chosen

        # Give new agents basic income
        person_leu = basic_income

        # Wealth is in Encointer currency
        person_wealth = person_chf * exchange_rate + person_leu

        # Version 1: Independent product price, product amount and production speed
        '''
        person_product_price = random.randint(5,10)
        person_product_amount = random.randint(50,100)
        person_production_speed = random.randint(10,20)
        '''
        # Version 2: Product price, product amount and production speed depends on social class
        if person_status_initial == 0:
            person_product_price = 0
            person_product_amount = 0
            person_production_speed = 0
        elif person_status_initial == 1:
            person_product_price = random.randint(5,10)
            person_product_amount = random.randint(50,100)
            person_production_speed = random.randint(10,20)
        else:
            person_product_price = random.randint(5,10)
            person_product_amount = random.randint(50,100)
            person_production_speed = random.randint(10,20)

        graph.add_node(person, chf_account = person_chf, leu_account = person_leu, wealth = person_wealth, product_amount = person_product_amount, product_price = person_product_price, production_speed = person_production_speed, initial_status = person_status_initial)

    #creating edges from new nodes to all nodes
    for person in range(last_person_index, last_person_index+amount_new_people):
        for other_person in range(0, last_person_index+amount_new_people):
            if person != other_person:
                person_prefer = random.randint(1, 10)
                graph.add_edge(person, other_person, prefer = person_prefer)

    #creating edges from old nodes to new nodes
    for person in range(0, last_person_index):
        for other_person in range(last_person_index, last_person_index+amount_new_people):
            if person != other_person:
                person_prefer = random.randint(1, 10)
                graph.add_edge(person, other_person, prefer = person_prefer)

### Initial States and System Parameters

In [ ]:
# cadCAD Simulation Parameters
SIMULATION_TIMESTEPS = 100#1825
MONTE_CARLO_RUNS = 1

In [ ]:
# Generate empty digraph
economy = networkx.MultiDiGraph()

In [ ]:
initial_state = {
    'economy': economy,
    'actions' : "-",

    # Plot for inequality
    'gini_index_wealth': gini(economy, 'wealth'),

    # Plots for products purchases
    'product_purchase_total': 0,
    'product_purchase_chf': 0,
    'product_purchase_leu': 0,
    'product_purchase_lower_class': 0,
    'product_purchase_middle_class': 0,
    'product_purchase_upper_class': 0,

    # Plots for demurrage and token supply
    'cum_demurrage_fee': 0,
    'money_minted_period': 0,
    'demurrage_fee_period' : 0,

    'simulation_type': None,
}

In [ ]:
system_params = {

    # Parameters for 2 runs: [<with Encointer>, <without Encointer>]
    'simulation_type': ['WITH_ENCOINTER', 'WITHOUT_ENCOINTER'],
    'demurrage': [demurrage(730), 0],
    'ceremony_interval': [5, SIMULATION_TIMESTEPS+1],
    'basic_income': [250, 0],
    'initial_population': [5, 50], #[> 1/newbie ratio] for encointer, [<max population>] without encointer
    'max_population': [50, 50], #max population that can join the system

    # Parameters for Population
    'lower_class_chf': [(0,0)],
    'middle_class_chf': [(500, 1500)],
    'upper_class_chf': [(0,0)],
    'lower_middle_boundary': [0],
    'middle_upper_boundary': [0],
    'percentage_of_classes': [(0.5, 0.5, 0)], #must add to 1

    # Parameter for Ceremony
    'newbie_ratio': [1/3], #newbie ratio of the ceremony participants

    # Parameters for Purchase and Sale of Products
    'max_nr_of_purchases_by_person': [5], #per day
    'preference_change_big':[10],

    # Parameter to change Preferences of an Agent over Time
    'preference_change_small':[1],

    # Parameter for Production Speed
    'max_product_stock': [200], #stop manufacture of products when above

    # Parameter for Exchange Rate
    'exchange_rate': [0.01], # LEU = CHF * exchange_rate
}

### POLICY FUNCTIONS
Description of the change from the current state (= previous state) to update the next state.

#### Substep 0

Bootstrapping

In [ ]:
def bootstrapping(params, substep, state_history, previous_state):

    if len(state_history) == 1:
        economy = previous_state['economy']
        initial_population = params['initial_population']
        basic_income = params['basic_income']
        percentage_of_classes = params['percentage_of_classes']
        lower_class_chf = params['lower_class_chf']
        middle_class_chf = params['middle_class_chf']
        upper_class_chf = params['upper_class_chf']
        exchange_rate = params['exchange_rate']

        generate_new_agents(economy, initial_population, basic_income, percentage_of_classes, lower_class_chf, middle_class_chf, upper_class_chf, exchange_rate)

        actions = "Bootstrapping the Encointer Community!"

    else:
        actions = "It is a new Day!"

    policy_input = {
        'actions': actions
    }

    return policy_input

#### Substep 1

Ceremony

In [ ]:
def ceremony(params, substep, state_history, previous_state):

    economy = previous_state['economy']
    ceremony_interval = params['ceremony_interval']
    basic_income = params['basic_income']
    max_population = params['max_population']
    newbie_ratio = params['newbie_ratio']

    percentage_of_classes = params['percentage_of_classes']
    lower_class_chf = params['lower_class_chf']
    middle_class_chf = params['middle_class_chf']
    upper_class_chf = params['upper_class_chf']
    lower_middle_boundary = params['lower_middle_boundary']
    middle_upper_boundary = params['middle_upper_boundary']
    exchange_rate = system_params['exchange_rate'][0]
    money_minted_period = previous_state['money_minted_period']

    leu_tuples = []
    wealth_tuples = []
    actions = []

    if len(state_history) % ceremony_interval != 0:
        actions = "No ceremony today!"
        money_minted_period = 0

    # Ceremony takes place
    if len(state_history) % ceremony_interval == 0:
        #print(f'It is ceremony time in timestep {len(state_history)}.')
        actions.append("Ceremony participants: index")

        wealth_of_agents = [economy.nodes[person_index]['wealth'] for person_index in economy.nodes]
        #print(f'Wealth {wealth_of_agents}.')
        max_wealth = max(wealth_of_agents)

        # Ceremony attendance

        probabilities_ceremony_participants = []
        for wealth_of_agent in wealth_of_agents:
            probabilities_ceremony_participants.append(find_probability_ceremony_participant(wealth_of_agent, max_wealth))
        #print(f'Probabilities ceremony participants {probabilities_ceremony_participants}.')

        bernoulli_results=[] #contains 0 or 1
        for probability in probabilities_ceremony_participants:
            bernoulli_results.append(np.random.binomial(1, probability, size=None))
        #print(f'Bernoulli_results {bernoulli_results}.')

        ceremony_participants =[person_index for person_index in economy.nodes if bernoulli_results[person_index] == 1]
        #print(f'Ceremony participants {ceremony_participants}.')

        for person_index in ceremony_participants:
            leu_tuples.append((person_index, basic_income))
            wealth_tuples.append((person_index, basic_income))

            if economy.nodes[person_index]['chf_account'] + economy.nodes[person_index]['leu_account'] / exchange_rate < lower_middle_boundary:
                actions.append(f"LOWER {person_index+1}")
            elif economy.nodes[person_index]['chf_account'] + economy.nodes[person_index]['leu_account'] / exchange_rate < middle_upper_boundary:
                actions.append(f"MIDDLE {person_index+1}")
            else:
                actions.append(f"UPPER {person_index+1}")

        # Community growth through newbies

        current_population = len(economy.nodes)

        # Only add newbies, if community size below max population
        if current_population <= max_population:

            # Protocol ensures only a fraction of the ceremony participants are newbies
            upper_bound_protocol = math.floor(len(ceremony_participants)*newbie_ratio) #round down to next int

            # Population grows along a logistic curve
            upper_bound_logistic_growth = logistic_growth(current_population,max_population)

            newbies_amount = min(upper_bound_protocol, upper_bound_logistic_growth)

            last_person_index = len(list(economy.nodes))
            for person in range(last_person_index, last_person_index+newbies_amount):
                actions.append(f"NEWBIE {person_index+1}")

            # Generate newbies
            generate_new_agents(economy, newbies_amount, basic_income, percentage_of_classes, lower_class_chf, middle_class_chf, upper_class_chf, exchange_rate)

        money_minted_period = basic_income * (len(ceremony_participants)+newbies_amount)

    policy_input = {
        'leu_accounts': leu_tuples,
        'wealth_accounts': wealth_tuples,
        'actions': actions,
        'money_minted_period': money_minted_period
    }

    return policy_input

#### Substep 2

Purchase and Sale of Products


In [ ]:
def buy_products(params, substep, state_history, previous_state):

    economy = previous_state['economy']
    max_nr_of_purchases_by_person = params['max_nr_of_purchases_by_person']
    preference_change_big = params['preference_change_big']

    lower_class_chf = params['lower_class_chf']
    middle_class_chf = params['middle_class_chf']
    upper_class_chf = params['upper_class_chf']
    lower_middle_boundary = params['lower_middle_boundary']
    middle_upper_boundary = params['middle_upper_boundary']
    exchange_rate = params['exchange_rate']

    #counting the purchase of products per day
    product_purchase_total = 0
    product_purchase_chf = 0
    product_purchase_leu = 0
    product_purchase_lower_class = 0
    product_purchase_middle_class = 0
    product_purchase_upper_class = 0

    leu_tuples=[]
    chf_tuples =[]
    wealth_tuples=[]
    amount_tuples=[]
    actions_tuples =[]
    preference_triples=[]

    actions_tuples.append("Buying products: (buyer, seller)")

    planned_product_amounts = [economy.nodes[person_index]['product_amount'] for person_index in economy.nodes]
    planned_leu_accounts = [economy.nodes[person_index]['leu_account'] for person_index in economy.nodes]
    planned_chf_accounts = [economy.nodes[person_index]['chf_account'] for person_index in economy.nodes]

    wealth_of_agents = [economy.nodes[person_index]['wealth'] for person_index in economy.nodes]
    #print(f'Wealth {wealth_of_agents}.')
    max_wealth = max(wealth_of_agents)

    # Finding the buyers

    probabilities_buyers=[]
    for wealth_of_agent in wealth_of_agents:
      probabilities_buyers.append(find_probability_buyer(wealth_of_agent, max_wealth))
    #print(f'Probabilities buyers {probabilities_buyers}.')

    binomial_results=[]
    for probability in probabilities_buyers:
      binomial_results.append(np.random.binomial(max_nr_of_purchases_by_person, probability, size=None))
    #print(f'Binomial results {binomial_results}.')

    buyers = []
    for person_index in range(len(binomial_results)): # person_index = 0,1,2
      bernoulli_result = binomial_results[person_index] # bernoulli_result =0,2,2
      while bernoulli_result > 0:
          buyers.append(person_index)
          bernoulli_result-=1
    #print(f'Buyers {buyers}.')

    random.shuffle(buyers)

    for buying_person_index in buyers:

        # Selecting the product

        # Get the potential_sellers according to...
        #   Rule 1: Planned product amount is bigger than 0
        #   Rule 2: Product price of the seller must be smaller than the leu account OR chf account of buyer
        #   Rule 3: Buyer and seller must not be the same person
        potential_sellers = [
            person_index for person_index in economy.nodes
            if planned_product_amounts[person_index] > 0
            and (
                economy.nodes[person_index]['product_price'] <= planned_leu_accounts[buying_person_index]
                or economy.nodes[person_index]['product_price'] / exchange_rate <= planned_chf_accounts[buying_person_index]
            )
            and buying_person_index != person_index
        ]

        # No seller found for that buyer
        if len(potential_sellers) == 0:
            continue #with the next buyer

        # Contains all preferences from the buyer to all potential_sellers
        prefer_scores = [economy[buying_person_index][person_index][0]['prefer'] for person_index in potential_sellers]
        total_prefer_score = sum(prefer_scores)

        # No division with 0
        if total_prefer_score == 0:
            continue #with the next buyer

        # Contains probabilities with which the buyer prefers the product of the seller
        probabilities_select_seller = [prefer_score/total_prefer_score for prefer_score in prefer_scores]

        # Choose the seller
        selling_person_index = np.random.choice(potential_sellers , p=probabilities_select_seller)

        planned_product_amounts[selling_person_index]-=1

        # Decision if to buy with LEU or CHF
        if economy.nodes[selling_person_index]['product_price'] <= planned_leu_accounts[buying_person_index]: #enough LEU

            planned_leu_accounts[buying_person_index]-=economy.nodes[selling_person_index]['product_price']
            planned_leu_accounts[selling_person_index]+=economy.nodes[selling_person_index]['product_price']

            leu_tuples.append((selling_person_index, economy.nodes[selling_person_index]['product_price']))
            leu_tuples.append((buying_person_index, -economy.nodes[selling_person_index]['product_price']))
            wealth_tuples.append((selling_person_index, economy.nodes[selling_person_index]['product_price']))
            wealth_tuples.append((buying_person_index, -economy.nodes[selling_person_index]['product_price']))

            product_purchase_leu+=1

        elif economy.nodes[selling_person_index]['product_price'] / exchange_rate <= planned_chf_accounts[buying_person_index]: #enough CHF

            planned_chf_accounts[buying_person_index]-=economy.nodes[selling_person_index]['product_price'] / exchange_rate
            planned_chf_accounts[selling_person_index]+=economy.nodes[selling_person_index]['product_price']/ exchange_rate

            chf_tuples.append((selling_person_index, economy.nodes[selling_person_index]['product_price']/ exchange_rate))
            chf_tuples.append((buying_person_index, -economy.nodes[selling_person_index]['product_price']/ exchange_rate))
            wealth_tuples.append((selling_person_index, economy.nodes[selling_person_index]['product_price']))
            wealth_tuples.append((buying_person_index, -economy.nodes[selling_person_index]['product_price']))

            product_purchase_chf+=1


        # TWO VERSIONS TO COUNT AMOUNT OF PURCHASES ACCORDING TO SOCIAL CLASSES
        '''# 1ST: CHECK HOW MUCH CHF THIS PERSON HAS NOW
        if economy.nodes[person_index]['chf_account'] + economy.nodes[person_index]['leu_account'] / exchange_rate < lower_middle_boundary:
            product_purchase_lower_class+=1
        elif economy.nodes[person_index]['chf_account'] + economy.nodes[person_index]['leu_account'] / exchange_rate < middle_upper_boundary:
            product_purchase_middle_class+=1
        else:
            product_purchase_upper_class+=1'''
        # 2ND: CHECK HOW MUCH CHF THIS PERSON HAD IN THE BEGINNING
        if economy.nodes[buying_person_index]['initial_status'] == 0:
            product_purchase_lower_class+=1
        elif economy.nodes[buying_person_index]['initial_status'] == 1:
            product_purchase_middle_class+=1
        elif economy.nodes[buying_person_index]['initial_status'] == 2:
            product_purchase_upper_class+=1

        amount_tuples.append((selling_person_index,-1))
        preference_triples.append((buying_person_index, selling_person_index, -preference_change_big))
        actions_tuples.append((buying_person_index+1, selling_person_index+1))
        product_purchase_total+=1

    policy_input = {
        'chf_accounts': chf_tuples,
        'leu_accounts': leu_tuples,
        'wealth_accounts': wealth_tuples,

        'product_amounts': amount_tuples,
        'actions': actions_tuples,
        'preference_triples': preference_triples,

        'product_purchase_total': product_purchase_total,
        'product_purchase_chf': product_purchase_chf,
        'product_purchase_leu': product_purchase_leu,
        'product_purchase_lower_class': product_purchase_lower_class,
        'product_purchase_middle_class': product_purchase_middle_class,
        'product_purchase_upper_class': product_purchase_upper_class,
    }

    return policy_input

#### Substep 3

Demurrage Fee
Product Manufacture
Preference Change

In [ ]:
def product_manufacture(params, substep, state_history, previous_state):

    economy = previous_state['economy']

    max_product_stock = params['max_product_stock']

    # For each agent add the production_speed
    product_tuples = []
    for person_index in economy.nodes:
        product_increment = economy.nodes[person_index]['production_speed']

        product_amount = economy.nodes[person_index]['product_amount']
        if product_amount > max_product_stock:
            product_increment = 0

        product_tuples.append((person_index, product_increment))

    policy_input = {
        'product_amounts': product_tuples,
    }
    return policy_input

In [ ]:
def pay_demurrage_fee(params, substep, state_history, previous_state):

    economy = previous_state['economy']
    demurrage = params['demurrage'] #now it is a number, before it was a list

    # Demurrage fee = demurrage * account; round to 2nd decimal place
    leu_tuples = [(person_index, -round(demurrage*economy.nodes[person_index]['leu_account'],2)) for person_index in economy.nodes]
    wealth_tuples = [(person_index, -round(demurrage*economy.nodes[person_index]['leu_account'],2)) for person_index in economy.nodes]

    demurrage_fee_per_day = 0
    for person_index in economy.nodes:
        demurrage_fee_per_day += round(demurrage*economy.nodes[person_index]['leu_account'], 2)

    # Cumulated demurrage fees of all agents
    cum_demurrage_fee = previous_state['cum_demurrage_fee'] + demurrage_fee_per_day

    # Sum up all demurrage fees between two ceremonies
    ceremony_interval = params['ceremony_interval']
    if len(state_history) % ceremony_interval == 1: #there was a ceremony yesterday
        demurrage_fee_period = demurrage_fee_per_day #reset demurrage period
    else:
        demurrage_fee_period = previous_state['demurrage_fee_period']+demurrage_fee_per_day

    policy_input = {
        'leu_accounts': leu_tuples,
        'wealth_accounts' : wealth_tuples,
        'cum_demurrage_fee': cum_demurrage_fee,
        'demurrage_fee_period': demurrage_fee_period
    }
    return policy_input

In [ ]:
def preference_change(params, substep, state_history, previous_state):

    economy = previous_state['economy']
    preference_change_small = params['preference_change_small']

    preference_triples = []

    #for each edge between two people increase the preference
    for buy_person_index in economy.nodes:
        for sell_person_index in economy.nodes:
            if buy_person_index != sell_person_index:
                preference_triples.append((buy_person_index, sell_person_index, preference_change_small))

    policy_input = {
        'preference_triples' : preference_triples
    }
    return policy_input

In [ ]:
def set_actions_demurrage(params, substep, state_history, previous_state):

    policy_input = {
        'actions' : "Demurrage fee, product manufacture & preference change"
    }
    return policy_input

#### Substep 4

Inequality measure

In [ ]:
def calculate_gini_index(params, substep, state_history, previous_state):

    gini_economy = previous_state['economy']

    policy_input = {
        #'gini_index_leu' : gini(gini_economy, 'leu_account'),
        #'gini_index_chf' : gini(gini_economy, 'chf_account'),
        'gini_index_wealth' : gini(gini_economy, 'wealth')
    }

    return policy_input

In [ ]:
def set_actions_gini(params, substep, state_history, previous_state):

    policy_input = {
        'actions' : "Gini index calculation"
    }
    return policy_input

### STATE UPDATE FUNCTIONS
Functions to update the state variables

In [ ]:
def update_economy(params, substep, state_history,  previous_state, policy_input):

    economy = previous_state['economy']

    # UPDATE LEU ACCOUNTS
    if 'leu_accounts' in policy_input:
        leu_tuples = policy_input['leu_accounts']
        for (person_index, account_movement) in leu_tuples:
            economy.nodes[person_index]['leu_account'] += account_movement

            # Python has a rounding problem (e.g. 0.48-0.47), therefore add another round
            economy.nodes[person_index]['leu_account'] = round(economy.nodes[person_index]['leu_account'],2)

    # UPDATE CHF ACCOUNTS
    if 'chf_accounts' in policy_input:
        chf_tuples = policy_input['chf_accounts']
        for (person_index, chf_account_movement) in chf_tuples:
            economy.nodes[person_index]['chf_account'] += chf_account_movement

            # Python has a rounding problem (e.g. 0.48-0.47), therefore add another round
            economy.nodes[person_index]['chf_account'] = round(economy.nodes[person_index]['chf_account'],2)

    # UPDATE WEALTH ACCOUNTS
    if 'wealth_accounts' in policy_input:
        wealth_tuples = policy_input['wealth_accounts']
        for (person_index, wealth_account_movement) in wealth_tuples:
            economy.nodes[person_index]['wealth'] += wealth_account_movement

            # Python has a rounding problem (e.g. 0.48-0.47), therefore add another round
            economy.nodes[person_index]['wealth'] = round(economy.nodes[person_index]['wealth'],2)

    # UPDATE PRODUCT AMOUNTS
    if 'product_amounts' in policy_input:
        product_amount_tuples = policy_input['product_amounts']
        for (person_index, product_amount_movement) in product_amount_tuples:
            economy.nodes[person_index]['product_amount'] += product_amount_movement


    # UPDATE PREFERENCES
    if 'preference_triples' in policy_input:
        preference_triples = policy_input['preference_triples']
        for (buy_person_index, sell_person_index, preference_change) in preference_triples:
            economy[buy_person_index][sell_person_index][0]['prefer'] += preference_change

            # Preference cannot be below 0
            if  economy[buy_person_index][sell_person_index][0]['prefer'] < 0:
                economy[buy_person_index][sell_person_index][0]['prefer'] = 0

    return 'economy', economy

In [ ]:
def update_actions(params, substep, state_history,  previous_state, policy_input):

    actions = policy_input['actions']

    return 'actions', actions

In [ ]:
def update_simulation_type(params, substep, state_history,  previous_state, policy_input):
    return 'simulation_type', params['simulation_type']

In [ ]:
def update_gini_index_wealth(params, substep, state_history,  previous_state, policy_input):
    gini_index_wealth = policy_input['gini_index_wealth']
    return 'gini_index_wealth', gini_index_wealth

In [ ]:
def update_product_purchase_total(params, substep, state_history,  previous_state, policy_input):
    product_purchase_total = policy_input['product_purchase_total']
    return 'product_purchase_total', product_purchase_total

def update_product_purchase_chf(params, substep, state_history,  previous_state, policy_input):
    product_purchase_chf = policy_input['product_purchase_chf']
    return 'product_purchase_chf', product_purchase_chf

def update_product_purchase_leu(params, substep, state_history,  previous_state, policy_input):
    product_purchase_leu = policy_input['product_purchase_leu']
    return 'product_purchase_leu', product_purchase_leu


def update_product_purchase_lower_class(params, substep, state_history,  previous_state, policy_input):
    product_purchase_lower_class = policy_input['product_purchase_lower_class']
    return 'product_purchase_lower_class', product_purchase_lower_class

def update_product_purchase_middle_class(params, substep, state_history,  previous_state, policy_input):
    product_purchase_middle_class = policy_input['product_purchase_middle_class']
    return 'product_purchase_middle_class', product_purchase_middle_class

def update_product_purchase_upper_class(params, substep, state_history,  previous_state, policy_input):
    product_purchase_upper_class = policy_input['product_purchase_upper_class']
    return 'product_purchase_upper_class', product_purchase_upper_class

In [ ]:
def update_product_purchase_total(params, substep, state_history,  previous_state, policy_input):
    product_purchase_total = policy_input['product_purchase_total']
    return 'product_purchase_total', product_purchase_total

def update_product_purchase_chf(params, substep, state_history,  previous_state, policy_input):
    product_purchase_chf = policy_input['product_purchase_chf']
    return 'product_purchase_chf', product_purchase_chf

def update_product_purchase_leu(params, substep, state_history,  previous_state, policy_input):
    product_purchase_leu = policy_input['product_purchase_leu']
    return 'product_purchase_leu', product_purchase_leu


def update_product_purchase_lower_class(params, substep, state_history,  previous_state, policy_input):
    product_purchase_lower_class = policy_input['product_purchase_lower_class']
    return 'product_purchase_lower_class', product_purchase_lower_class

def update_product_purchase_middle_class(params, substep, state_history,  previous_state, policy_input):
    product_purchase_middle_class = policy_input['product_purchase_middle_class']
    return 'product_purchase_middle_class', product_purchase_middle_class

def update_product_purchase_upper_class(params, substep, state_history,  previous_state, policy_input):
    product_purchase_upper_class = policy_input['product_purchase_upper_class']
    return 'product_purchase_upper_class', product_purchase_upper_class

In [ ]:
def update_cum_demurrage_fee(params, substep, state_history,  previous_state, policy_input):
    cum_demurrage_fee = policy_input['cum_demurrage_fee']
    return 'cum_demurrage_fee', cum_demurrage_fee

def update_money_minted_period(params, substep, state_history,  previous_state, policy_input):
    money_minted_period = policy_input['money_minted_period']
    return 'money_minted_period', money_minted_period

def update_demurrage_fee_period(params, substep, state_history,  previous_state, policy_input):
    demurrage_fee_period = policy_input['demurrage_fee_period']
    return 'demurrage_fee_period', demurrage_fee_period

### PARTIAL STATE UPDATE BLOCKS

A series of Partial State Update Blocks is a structure for composing State Update Functions and Policy Functions in series or parallel, as a representation of the system model.

In [ ]:
partial_state_update_blocks = [
    #substep 0
    {
        'policies': {
            'bootstrapping': bootstrapping,
        },
        'variables': {
            'economy': update_economy,
            'actions': update_actions,
        }
    },
    #substep 1
    {
        'policies': {
            'ceremony': ceremony,
        },
        'variables': {
            'economy': update_economy,
            'actions': update_actions,
            'simulation_type': update_simulation_type,
            'money_minted_period': update_money_minted_period
        }
    },
    #substep 2
    {
        'policies': {
            'buy_products': buy_products,
        },
        'variables': {
            'economy': update_economy,
            'actions': update_actions,

            #plots
            'product_purchase_total': update_product_purchase_total,
            'product_purchase_chf': update_product_purchase_chf,
            'product_purchase_leu': update_product_purchase_leu,
            'product_purchase_middle_class': update_product_purchase_middle_class,
            'product_purchase_lower_class': update_product_purchase_lower_class,
            'product_purchase_upper_class': update_product_purchase_upper_class,
        }
    },
    #substep 3
    {
        'policies': {
            'decrease_accounts': pay_demurrage_fee,
            'increase_product_amounts': product_manufacture,
            'increase_preferences': preference_change,
            'set_actions_demurrage': set_actions_demurrage,
        },
        'variables': {
            'economy': update_economy,
            'actions': update_actions,
            'cum_demurrage_fee': update_cum_demurrage_fee,
            'demurrage_fee_period': update_demurrage_fee_period
        }
    },
    #substep 4
    {
        'policies': {
            'calculate_gini_index': calculate_gini_index,
            'set_actions_gini': set_actions_gini,
        },
        'variables': {
            'gini_index_wealth': update_gini_index_wealth,
            'actions': update_actions,
        }
    }
]

### CONFIGURATION & EXECUTION

In [ ]:
# CADCAD configuration
sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params
    }
)

In [ ]:
from cadCAD import configs

# Clear any prior configs
del configs[:]

experiment = Experiment()

# Change configs from 'import configs'
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks =partial_state_update_blocks
)

exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs) #the table is created here

# Table is stored in system_events
(system_events, tensor_field, sessions) = run.execute()

### OUTPUT PREPARATION

In [ ]:
# Modify system_events

for system_event in system_events:

    economy = system_event['economy']
    exchange_rate = system_params['exchange_rate'][0]

    lower_middle_boundary = system_params['lower_middle_boundary'][0]
    middle_upper_boundary = system_params['middle_upper_boundary'][0]

    chf_accounts = [economy.nodes[person_index]['chf_account'] for person_index in economy.nodes]
    system_event['chf_accounts'] = chf_accounts

    leu_accounts = [economy.nodes[person_index]['leu_account'] for person_index in economy.nodes]
    system_event['leu_accounts'] = leu_accounts

    wealth = [economy.nodes[person_index]['wealth'] for person_index in economy.nodes]
    system_event['wealth'] = wealth

    initial_status = [economy.nodes[person_index]['initial_status'] for person_index in economy.nodes]
    system_event['status_initial'] = initial_status

    product_amounts = [economy.nodes[person_index]['product_amount'] for person_index in economy.nodes]
    system_event['product_amounts'] = product_amounts

    product_prices = [economy.nodes[person_index]['product_price'] for person_index in economy.nodes]
    system_event['product_prices'] = product_prices

    production_speeds = [economy.nodes[person_index]['production_speed'] for person_index in economy.nodes]
    system_event['production_speeds'] = production_speeds

    preferences=[]
    for buyer_index in economy.nodes:
        helper_list=[]
        for seller_index in economy.nodes:
            #no preference for itself
            if seller_index == buyer_index:
                helper_list.append('-')
                continue #with next buyer
            helper_list.append(economy[buyer_index][seller_index][0]['prefer'])
        preferences.append(helper_list)
    system_event['preferences'] = preferences

    # Community size
    number_of_people = len(list(economy.nodes))
    system_event['community_size'] = number_of_people

    # Number of people
    number_of_lower_class_people = 0
    number_of_middle_class_people = 0
    number_of_upper_class_people = 0
    for person_index in economy.nodes:
        if economy.nodes[person_index]['chf_account'] + economy.nodes[person_index]['leu_account'] / exchange_rate < lower_middle_boundary:
            number_of_lower_class_people += 1
        elif economy.nodes[person_index]['chf_account'] + economy.nodes[person_index]['leu_account'] / exchange_rate < middle_upper_boundary:
            number_of_middle_class_people += 1
        else:
            number_of_upper_class_people += 1
    system_event['lower_class_people'] = number_of_lower_class_people
    system_event['middle_class_people'] = number_of_middle_class_people
    system_event['upper_class_people'] = number_of_upper_class_people

    sum_of_product_amounts = 0
    for product_amount in product_amounts:
        sum_of_product_amounts += product_amount
    system_event['sum_of_product_amounts'] = sum_of_product_amounts

    sum_of_leu_accounts = 0
    for leu_account in leu_accounts:
        sum_of_leu_accounts += leu_account
    system_event['leu_total'] = sum_of_leu_accounts

    sum_of_chf_accounts = 0
    for chf_account in chf_accounts:
        sum_of_chf_accounts += chf_account
    system_event['sum_of_chf_accounts'] = sum_of_chf_accounts

    sum_of_wealths = 0
    for person_wealth in wealth:
        sum_of_wealths += person_wealth
    system_event['wealth_total'] = sum_of_wealths

    del system_event['simulation']
    del system_event['subset']

### PLOT THE TABLE

In [ ]:
# Use pandas library to create a table with the information stored in system_events (= display of the data)
df = pd.DataFrame(system_events)
df = df.set_index(['run','timestep','substep']) #set the index
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
df

In [ ]:
#save into an Excel spreadsheet
df.to_excel('results_encointer_simulation/encointer_simulation_table.xlsx')

print('cadCAD has executed the simulation successfully!')

#close excel table before running it again!

### PLOT THE NETWORKX GRAPH

In [ ]:
# Print as NetworkX graph

# Get the agents balance
G = system_events[0]['economy'] #insert timestep into []
node_balances = networkx.get_node_attributes(G, 'account')
node_labels = {node: f"{value :.0f}"
    for node, value
    in node_balances.items()}

# Node sizes
sizes = list(node_balances.values())

# Prepare the figure
matplotlib.pyplot.figure(figsize=(12, 4))

# Draw the nodes
pos = networkx.spring_layout(G)
networkx.draw(G,
    pos,
    node_size=sizes)

# Draw the balances
networkx.draw_networkx_labels(G, pos, labels=node_labels)

# Show the visualization
matplotlib.pyplot.show()

### PLOT THE GRAPHS

In [ ]:
# Save the dataframe as system_events_table
system_events_table = df

# Reset the indexes (since indexes are "run, timestep, substep")
system_events_table = system_events_table.reset_index()

#### WITH ENCOINTER FRAMEWORK

In [ ]:
# Set a mask for Encointer and filter all substeps 5
mask = (system_events_table['simulation_type']=='WITH_ENCOINTER') & (system_events_table['substep']==5) #| (system_events_table['substep']==0)
system_events_with_encointer_table = system_events_table[mask]

# Set the new indexes
system_events_with_encointer_table = system_events_with_encointer_table.set_index('timestep')

In [ ]:
# PLOT COMMUNITY SIZE

community_size_table = system_events_with_encointer_table[['community_size']]
community_size_table.rename(columns = {'community_size': 'Community Size'}, inplace = True)

matplotlib.pyplot.figure('Community Size - Encointer')

axes=seaborn.lineplot(data = community_size_table)
axes.set_ylim(ymin=0)
axes.set_xlim(1, 1825)

# Set grid on x-axis on ceremony timesteps
#axes.set_xticks(ceremony_list)

matplotlib.pyplot.xlabel('Time (Days)')
matplotlib.pyplot.ylabel('Number of People')

matplotlib.pyplot.xticks(rotation=90)

matplotlib.pyplot.savefig('results_encointer_simulation/Community Size - Encointer', dpi=300, bbox_inches = "tight")

In [ ]:
# PLOT LEU IN CIRCULATION AND DEMURRAGE

leu_total_demurrage_fee_table = system_events_with_encointer_table[['cum_demurrage_fee', 'leu_total']]

leu_total_demurrage_fee_table.rename(columns = {'leu_total': 'Total OGU Supply Improved'}, inplace = True)
leu_total_demurrage_fee_table.rename(columns = {'cum_demurrage_fee': 'Cumulated Demurrage Improved'}, inplace = True)


matplotlib.pyplot.figure('OGU in Circulation and Demurrage - Encointer')

axes=seaborn.lineplot(data = leu_total_demurrage_fee_table)

#set grid on x-axis on ceremony timesteps
#axes.set_xticks(ceremony_list)

#name the axes
matplotlib.pyplot.xlabel('Time (Days)')
matplotlib.pyplot.ylabel('OGU')
axes.set_xlim(1, 1825)
axes.set_ylim(1, 1000000)

matplotlib.pyplot.xticks(rotation=90)

matplotlib.pyplot.savefig('results_encointer_simulation/OGU in Circulation and Demurrage - Encointer', dpi=300, bbox_inches = "tight")

In [ ]:
# PLOT OGU MINTED AND DEMURRAGE BETWEEN TWO CEREMONIES

demurrage_minted_money = system_events_with_encointer_table[['demurrage_fee_period', 'money_minted_period']]

demurrage_minted_money.rename(columns = {'money_minted_period': 'Total OGU in Ceremony Improved'}, inplace = True)
demurrage_minted_money.rename(columns = {'demurrage_fee_period': 'Total Demurrage between Ceremonies Improved'}, inplace = True)


#returns cumulative sum over a DataFrame or Series

matplotlib.pyplot.figure('Demurrage and Money Minted Between 2 Ceremonies - Encointer')

axes=seaborn.lineplot(data = demurrage_minted_money)

#set grid on x-axis on ceremony timesteps
#axes.set_xticks(ceremony_list)

#name the axes
matplotlib.pyplot.xlabel('Time (Days)')
matplotlib.pyplot.ylabel('OGU')
axes.set_xlim(1, 1825)
axes.set_ylim(1, 6000)

matplotlib.pyplot.xticks(rotation=90)
matplotlib.pyplot.savefig('results_encointer_simulation/Demurrage and Money Minted Between 2 Ceremonies', dpi=300, bbox_inches = "tight")

#### COMPARISON ENCOINTER AND NO ENCOINTER FRAMEWORK


In [ ]:
mask = (system_events_table['simulation_type']=='WITHOUT_ENCOINTER') & (system_events_table['substep']==5)
system_events_without_encointer_table = system_events_table[mask]

#set the new indexes
system_events_without_encointer_table = system_events_without_encointer_table.set_index('timestep')

In [ ]:
# PLOT GINI INDEX FOR CHF AND WEALTH

with_encointer_gini_table = system_events_with_encointer_table[['gini_index_wealth']]
without_encointer_gini_table = system_events_without_encointer_table[['gini_index_wealth']]

gini_table = pd.DataFrame(index=system_events_with_encointer_table.index)

gini_table['Gini Index w/o OGU Improved'] = system_events_without_encointer_table['gini_index_wealth']
gini_table['Gini Index w/ OGU Improved'] = system_events_with_encointer_table['gini_index_wealth']

matplotlib.pyplot.figure('Gini Indexes - Encointer and NO Encointer')

axes=seaborn.lineplot(data = gini_table)
axes.set_ylim(0, 1)
axes.set_xlim(1, 1825)

matplotlib.pyplot.xlabel('Time (Days)')
matplotlib.pyplot.ylabel('Gini Index')

matplotlib.pyplot.xticks(rotation=90)

matplotlib.pyplot.savefig('results_encointer_simulation/Gini Indexes - Encointer and NO Encointer',dpi = 300, bbox_inches = "tight")

In [ ]:
# DAILY PRODUCT PURCHASES

with_encointer_trade_product_table = system_events_with_encointer_table[['product_purchase_total', 'product_purchase_lower_class', 'product_purchase_middle_class', 'product_purchase_upper_class']]
without_encointer_trade_product_table = system_events_without_encointer_table[['product_purchase_total', 'product_purchase_lower_class', 'product_purchase_middle_class', 'product_purchase_upper_class']]

In [ ]:
# PLOT PRODUCT PURCHASE DAILY FOR SYSTEM WITH AND WITHOUT ENCOINTER

trade_product_table = pd.DataFrame(index=system_events_with_encointer_table.index)
#trade_product_table['with_encointer_product_purchase_daily'] = with_encointer_trade_product_table['product_purchase_total']
#trade_product_table['without_encointer_product_purchase_daily'] = without_encointer_trade_product_table['product_purchase_total']

trade_product_table['Total Product Purchases Daily w/o OGU Improved'] = without_encointer_trade_product_table['product_purchase_total']
trade_product_table['Total Product Purchases Daily with OGU Improved'] = with_encointer_trade_product_table['product_purchase_total']


matplotlib.pyplot.figure('Product Trade Daily - Total - Encointer and NO Encointer')

axes=seaborn.lineplot(data = trade_product_table)

matplotlib.pyplot.xlabel('Time (Days)')
matplotlib.pyplot.ylabel('Amount of Products')
axes.set_xlim(1, 1825)

matplotlib.pyplot.xticks(rotation=90)
matplotlib.pyplot.savefig('results_encointer_simulation/Product Trade Daily - Total - Encointer and NO Encointer', dpi=300, bbox_inches = "tight")

In [ ]:
# PLOT AVERAGE PRODUCT PURCHASE PER PERSON FOR SYSTEM WITHOUT ENCOINTER

trade_product_table = pd.DataFrame(index=system_events_with_encointer_table.index)

trade_product_table['Avg. Daily Products Purchases per Person w/o OGU Improved'] = without_encointer_trade_product_table['product_purchase_total'] / 50

matplotlib.pyplot.figure('Product Trade Daily - Per Person - NO Encointer')

axes=seaborn.lineplot(data = trade_product_table)

y = [0,1,2,3,4,5]
matplotlib.pyplot.yticks(y)
axes.set_xlim(1, 1825)

matplotlib.pyplot.xlabel('Time (Days)')
matplotlib.pyplot.ylabel('Amount of Products')

matplotlib.pyplot.xticks(rotation=90)
matplotlib.pyplot.savefig('results_encointer_simulation/Product Trade Daily - Per Person - NO Encointer', dpi=300, bbox_inches = "tight")

In [ ]:
# PLOT PRODUCT PURCHASE DAILY FOR SYSTEM WITH AND WITHOUT ENCOINTER

with_encointer_trade_product_table = system_events_with_encointer_table[['product_purchase_total', 'product_purchase_leu', 'product_purchase_chf']]


trade_product_table = pd.DataFrame(index=system_events_with_encointer_table.index)

#trade_product_table['Product Purchases without OGU in Total'] = without_encointer_trade_product_table['product_purchase_total']
trade_product_table['Product Purchases in Total'] = with_encointer_trade_product_table['product_purchase_total']
trade_product_table['Product Purchases in OGU'] = with_encointer_trade_product_table['product_purchase_leu']
trade_product_table['Product Purchases in CFA-Franc'] = with_encointer_trade_product_table['product_purchase_chf']

matplotlib.pyplot.figure('Product Trade Daily with OGU - Total, OGU, CFA-Franc')

axes=seaborn.lineplot(data = trade_product_table)

axes.set_xlim(1, 1825)

matplotlib.pyplot.xlabel('Time [Days]')
matplotlib.pyplot.ylabel('Amount of Products')

matplotlib.pyplot.xticks(rotation=90)
matplotlib.pyplot.savefig('results_encointer_simulation/Product Trade Daily with OGU - Total, OGU, CFA-Franc', dpi=300, bbox_inches = "tight")

In [ ]:
without_encointer_trade_product_table = system_events_without_encointer_table[['product_purchase_total', 'product_purchase_lower_class', 'product_purchase_middle_class']]

trade_product_table = pd.DataFrame(index=system_events_with_encointer_table.index)

trade_product_table['Product Purchases in Total Improved'] = without_encointer_trade_product_table['product_purchase_total']
trade_product_table['Product Purchases of Indigents Improved'] = without_encointer_trade_product_table['product_purchase_lower_class']


matplotlib.pyplot.figure('Product Trade Daily without OGU - Social Class')

axes=seaborn.lineplot(data = trade_product_table)

axes.set_xlim(1, 1825)
axes.set_ylim(0, 250)

matplotlib.pyplot.xlabel('Time [Days]')
matplotlib.pyplot.ylabel('Amount of Products')

matplotlib.pyplot.xticks(rotation=90)
matplotlib.pyplot.savefig('results_encointer_simulation/Product Trade Daily without OGU - Social Class', dpi=300, bbox_inches = "tight")

In [ ]:
with_encointer_trade_product_table = system_events_with_encointer_table[['product_purchase_total', 'product_purchase_lower_class', 'product_purchase_middle_class']]

trade_product_table = pd.DataFrame(index=system_events_with_encointer_table.index)

#trade_product_table['Product Purchases without OGU in Total'] = without_encointer_trade_product_table['product_purchase_total']
trade_product_table['Total Product Purchases w/ OGU Improved'] = with_encointer_trade_product_table['product_purchase_total']
trade_product_table['Product Purchases of Indigents w/ OGU Improved'] = with_encointer_trade_product_table['product_purchase_lower_class']

matplotlib.pyplot.figure('Product Trade Daily with OGU - Social Class')

axes=seaborn.lineplot(data = trade_product_table)

axes.set_xlim(1, 1825)
axes.set_ylim(0, 250)

matplotlib.pyplot.xlabel('Time [Days]')
matplotlib.pyplot.ylabel('Amount of Products')

matplotlib.pyplot.xticks(rotation=90)
matplotlib.pyplot.savefig('results_encointer_simulation/Product Trade Daily with OGU - Social Class', dpi=300, bbox_inches = "tight")